In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/shap-dim-reduction/shap-dim-reduction"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/shap-dim-reduction/shap-dim-reduction


In [ ]:
try:
  ! git config --global user.email "parkervg5@gmail.com"
  ! git config --global user.name "parkervg"
  ! git pull
except:
  ! git stash 
  ! git pull

Already up to date.


In [ ]:
try:
  import shap 
except:
  !pip install shap

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import bz2
import re
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.linear_model import LogisticRegression
from lib.ProcessEmbeddings import WordEmbeddings
from sklearn.utils import shuffle

2020-11-10 01:41:47 INFO     'pattern' package not found; tag filters are not available for English


In [ ]:
DIMS = 50

##Definition of Functions

In [ ]:
def get_sent_vectors(sents, word_vectors, wvec_dim):
  """
  Averages over all word vectors in a sentence.
  """
  embeddings = []
  for sent in sents:
      sentvec = []
      for word in word_tokenize(sent):
        try:
            sentvec.append(word_vectors[word.lower()])
        except:
            pass
      if not sentvec:
          vec = np.zeros(wvec_dim)
          sentvec.append(vec)
      sentvec = np.mean(sentvec, 0)
      embeddings.append(sentvec)
  embeddings = np.vstack(embeddings)
  return embeddings

In [ ]:
def standard_glove():
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return get_sent_vectors(train_text, word_vectors, wvec_dim), get_sent_vectors(test_text, word_vectors, wvec_dim)

In [ ]:
def algo_glove(output_dims):
  """
  Following the "Algo-N" process described in https://www.aclweb.org/anthology/W19-4328/
  """
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  # PPE
  WE.subract_mean()
  WE.pca_fit()
  WE.remove_top_components(k=7)

  # PCA dim reduction
  WE.subract_mean()
  WE.pca_fit_transform(output_dims=output_dims)

  # PPE
  WE.subract_mean()
  WE.pca_fit()
  WE.remove_top_components(k=7)
  
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return get_sent_vectors(train_text, word_vectors, wvec_dim), get_sent_vectors(test_text, word_vectors, wvec_dim)

In [ ]:
def shap_glove(task, output_dims):
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  dims = WE.shap_dim_reduction(task=task, k=output_dims)
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return get_sent_vectors(train_text, word_vectors, wvec_dim), get_sent_vectors(test_text, word_vectors, wvec_dim), dims

In [ ]:
def rand_dims_glove(k, avoid_dims=[]):
  WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
  dims = WE.rand_dim_reduction(k=k, avoid_dims=avoid_dims)
  word_vectors = WE.get_vector_dict()
  wvec_dim = WE.embeds.shape[1]
  return get_sent_vectors(train_text, word_vectors, wvec_dim), get_sent_vectors(test_text, word_vectors, wvec_dim), dims

In [ ]:
def labels_text(x):
  label=[]
  text=[]
  for line in bz2.BZ2File(x):
    decode = line.decode("utf-8")
    label.append(int(decode[9]) - 1)
    text.append(decode[10:].strip())
  return np.array(label),text

##Loading dataset

In [ ]:
train_label, train_text = labels_text('data/amazon-reviews/train.ft.txt.bz2')
test_label, test_text = labels_text('data/amazon-reviews/test.ft.txt.bz2')

# We're only using a subset of the dataset, so we shuffle it 
train_text, train_label = shuffle(train_text, train_label)
test_text, test_label = shuffle(test_text, test_label)

train_text = train_text[:10000]
train_label = train_label[:10000]
test_text = test_text[:5000]
test_label = test_label[:5000]

##Loading Processed Vectors

In [ ]:
Xtrain_glove, Xtest_glove = standard_glove()

Xtrain_algo, Xtest_algo = algo_glove(output_dims=DIMS)

Xtrain_shap_cr, Xtest_shap_cr, cr_dims = shap_glove(task="CR", output_dims=DIMS)

Xtrain_shap_mpqa, Xtest_shap_mpqa, mpqa_dims = shap_glove(task="MPQA", output_dims=DIMS)

Xtrain_shap_trec, Xtest_shap_trec, trec_dims = shap_glove(task="TREC", output_dims=DIMS)

2020-11-10 01:43:30 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-10 01:44:00 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-10 01:45:06 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-10 01:45:23 INFO     Found 5282 words with word vectors, out of             5677 words
2020-11-10 01:45:23 INFO     Generating sentence embeddings
2020-11-10 01:45:24 INFO     Generated sentence embeddings
2020-11-10 01:45:24 INFO     Training sklearn-LogReg with (inner) 5-fold cross-validation
2020-11-10 01:45:26 INFO     Best param found at split 1: l2reg = 2                 with score 78.01
2020-11-10 01:45:28 INFO     Best param found at split 2: l2reg = 2                 with score 77.45
2020-11-10 01:45:31 INFO     Best param found at split 3: l2reg = 4                 with score 78.71
2020-11-10 01:45:33 INFO     Best param found at split 4: l2reg = 4                 with score 78.34
2020-11-10 01:45:35 INFO     Best param found at split 5: l2reg 

Full 300 dimension Glove accuracy

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_glove, train_label)
print(round(clf.score(Xtest_glove, test_label)* 100, 2))

82.68


In [ ]:
# Fitting SHAP to the amazon-review dataset

WE = WordEmbeddings(vector_file="embeds/glove.6B.300d.txt")
glove_dims = WE.top_shap_dimensions(clf, Xtrain_glove, k=DIMS)
Xtrain_glove_reduced = np.take(Xtrain_glove, indices=glove_dims, axis=1)
Xtest_glove_reduced = np.take(Xtest_glove, indices=glove_dims, axis=1)

# Random sample dimensions as baseline
# With 300 dimensions and k = 150, basically just all 150 dimensions not in glove_dims
Xtrain_rand, Xtest_rand, rand_dims = rand_dims_glove(avoid_dims=glove_dims, k=DIMS)

2020-11-10 01:47:29 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-10 01:47:46 INFO     Classifier has 2 classes
2020-11-10 01:47:46 INFO     Loading vectors at embeds/glove.6B.300d.txt...
2020-11-10 01:48:04 INFO     Randomly selected dimension indices [101, 3, 197, 282, 234, 122, 82, 223, 115, 233, 7, 161, 159, 56, 172, 88, 279, 136, 195, 95, 27, 186, 121, 91, 176, 69, 89, 42, 235, 296, 138, 284, 65, 116, 35, 255, 263, 143, 231, 294, 155, 157, 0, 5, 180, 1, 33, 57, 54, 241]
2020-11-10 01:48:04 INFO     New shape of embeds is (400000, 50)


SHAP amazon-review reduced Glove accuracy

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_glove_reduced, train_label)
print(round(clf.score(Xtest_glove_reduced, test_label) * 100, 2))

79.8


Algo-N reduced Glove accuracy: 50 dimensions

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_algo, train_label)
print(round(clf.score(Xtest_algo, test_label) * 100, 2))

72.4


SHAP reduced accuracy on CR



In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_cr, train_label)
print(round(clf.score(Xtest_shap_cr, test_label) * 100, 2))

76.74


SHAP reduced on MPQA

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_mpqa, train_label)
print(round(clf.score(Xtest_shap_mpqa, test_label) * 100, 2))

75.8


SHAP reduced on TREC

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_shap_trec, train_label)
print(round(clf.score(Xtest_shap_trec, test_label) * 100, 2))

75.12


Randomly reduced 50 dimension accuracy

In [ ]:
clf = LogisticRegression()
clf.fit(Xtrain_rand, train_label)
print(round(clf.score(Xtest_rand, test_label) * 100, 2))

70.92


In [ ]:
def intersect(a, b):
  return len(list(set(a) & set(b)))
print(f"Intersecting dims between CR and amazon-review: {intersect(glove_dims, cr_dims)}")
print()
print(f"Intersecting dims between MR and amazon-review: {intersect(glove_dims, mpqa_dims)}")
print()
print(f"Intersecting dims between TREC and amazon-review: {intersect(glove_dims, trec_dims)}")
print()
print(f"Intersecting dims between random dims and amazon-review: {intersect(glove_dims, rand_dims)}")

Intersecting dims between CR and amazon-review: 18

Intersecting dims between MR and amazon-review: 13

Intersecting dims between TREC and amazon-review: 11

Intersecting dims between random dims and amazon-review: 0
